## building the model

In [1]:
# importing data manipulation libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
# importing basic machine learning libraries
import keras as ks
import sklearn as sk
import tensorflow as tf

In [3]:
# importing ANN libraries
from keras import models
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from keras.layers import Activation
from keras.utils.generic_utils import get_custom_objects
from keras import regularizers
from sklearn.model_selection import KFold

In [4]:
# creating gaussian function for the RBF layer

def custom_gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'custom_gelu': Activation(custom_gelu)})

In [5]:
# creating the custom rmse for evaluation

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true)))

## data manipulation

In [6]:
inputs = pd.read_csv('Cass1.csv',delimiter=',', header=None) #loading inputs from a different file 
print (inputs.shape)
targets = pd.read_csv('UTM1.csv',delimiter=',', header=None) # loading targets
print(targets.shape)


(48, 2)
(48, 2)


In [7]:
inputf = pd.read_csv('Cass_1.csv',delimiter=',', header=None) #loading inputs from a different file 
print (inputf.shape)
target = pd.read_csv('UTM_1.csv',delimiter=',', header=None) # loading targets
print(target.shape)


(17, 2)
(17, 2)


In [8]:
X_trainB = inputs
y_trainB = targets
X_testB = inputf
y_testB = target
y_trainB.shape

(48, 2)

### pre-processing


In [9]:
all_data = np.concatenate((inputs, targets),axis=1)
X = all_data[0:,:2]
y = all_data[0:,2:]

In [10]:
all_data.shape

(48, 4)

In [11]:
# fitting max - min scaler

from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler() #scaler for the inputs
min_max_scaler.fit(X_trainB) #fitting the scaler

min_max_scaler_Target = preprocessing.MinMaxScaler() #scaler for targets
min_max_scaler_Target.fit(y_trainB)


MinMaxScaler()

In [12]:
# normalizing the training data

train_data_1 = min_max_scaler.transform(X_trainB)
train_targets_1 = min_max_scaler_Target.transform(y_trainB)
print (train_data_1.shape)
print (train_targets_1.shape)

(48, 2)
(48, 2)


In [13]:
# normalizing the testing data

test_data_1 = min_max_scaler.transform(X_testB)
test_targets_1 = min_max_scaler_Target.transform(y_testB)
print (test_data_1.shape)
print (test_targets_1.shape)

(17, 2)
(17, 2)


In [14]:
X_train = train_data_1
y_train = train_targets_1
X_test = test_data_1
y_test = test_targets_1

In [15]:
def linear(x):
    return x
get_custom_objects().update({'linear': Activation(linear)})

## Running the model

In [16]:
## Random seeding
np.random.seed(123)
tf.random.set_seed(123)
input_size=2
output_size=2

k = 3

num_val_samples = len(X_train) // k

num_epochs = 4200

all_scores =[]

labels =[]

prd = []
fold_val = []
fold_targ = []
all_mse =[]

for i in range(k):
    print('processing fold #', i)
    ## validation data comes from training data for each loop
    val_data = X_train[i * num_val_samples : (i +1)*num_val_samples]
    val_targets = y_train[i * num_val_samples : (i+1) * num_val_samples]
    ## creating the folds
    partial_train_data = np.concatenate([X_train[:i*num_val_samples], X_train[(i + 1) *num_val_samples :]], axis=0)
    partial_train_targets = np.concatenate([y_train[:i*num_val_samples], y_train[(i + 1) *num_val_samples :]], axis=0)         
    models = tf.keras.Sequential() 
    models.add(tf.keras.layers.Dense(63,  activation=linear,kernel_regularizer=regularizers.l2(0.000001), input_shape = (2,))),
    #models.add(tf.keras.layers.Dense(63)),
    models.add(tf.keras.layers.Dense(2)),
    models.build((2, 2)),
               
   
                            
    custom_optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
    models.compile(optimizer=custom_optimizer,loss='mean_squared_error')
# compiling the model 

    models.fit(partial_train_data, partial_train_targets, epochs = num_epochs, verbose=1)
    ## evaluate the mse, mae in validation data
  
    #val_mse, val_mae = models.evaluate(val_data, val_targets, verbose=1)
    ## Predict the test targets
    #preds = model.predict(X_test)
    ## evaluate on test datasets
  #  true_mse_mae = model.evaluate(test_data, test_targets)
    
    
    #all_scores.append(val_mae)
    #all_mse.append(val_mse)
    preds = models.predict(val_data)
    labels.append(val_targets)
    prd.append(preds)
    
    fold_val.append(val_data)
    fold_targ.append(val_targets)
    print (preds)


processing fold # 0
Epoch 1/4200
1/1 [==============================] - 0s 41ms/step - loss: 0.5167
Epoch 2/4200
1/1 [==============================] - 0s 3ms/step - loss: 0.2859
Epoch 3/4200
1/1 [==============================] - 0s 2ms/step - loss: 0.1465
Epoch 4/4200
1/1 [==============================] - 0s 2ms/step - loss: 0.0874
Epoch 5/4200
1/1 [==============================] - 0s 2ms/step - loss: 0.0867
Epoch 6/4200
1/1 [==============================] - 0s 2ms/step - loss: 0.1085
Epoch 7/4200
1/1 [==============================] - 0s 3ms/step - loss: 0.1206
Epoch 8/4200
1/1 [==============================] - 0s 2ms/step - loss: 0.1122
Epoch 9/4200
1/1 [==============================] - 0s 1000us/step - loss: 0.0889
Epoch 10/4200
1/1 [==============================] - 0s 3ms/step - loss: 0.0607
Epoch 11/4200
1/1 [==============================] - 0s 2ms/step - loss: 0.0362
Epoch 12/4200
1/1 [==============================] - 0s 2ms/step - loss: 0.0200
Epoch 13/4200
1/1 [======

1/1 [==============================] - 0s 2ms/step - loss: 7.0906e-06
Epoch 101/4200
1/1 [==============================] - 0s 1000us/step - loss: 8.2886e-06
Epoch 102/4200
1/1 [==============================] - 0s 3ms/step - loss: 8.5058e-06
Epoch 103/4200
1/1 [==============================] - 0s 2ms/step - loss: 7.5507e-06
Epoch 104/4200
1/1 [==============================] - 0s 2ms/step - loss: 5.9386e-06
Epoch 105/4200
1/1 [==============================] - 0s 2ms/step - loss: 4.4759e-06
Epoch 106/4200
1/1 [==============================] - 0s 4ms/step - loss: 3.7660e-06
Epoch 107/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.9278e-06
Epoch 108/4200
1/1 [==============================] - 0s 999us/step - loss: 4.6276e-06
Epoch 109/4200
1/1 [==============================] - 0s 2ms/step - loss: 5.3342e-06
Epoch 110/4200
1/1 [==============================] - 0s 2ms/step - loss: 5.6217e-06
Epoch 111/4200
1/1 [==============================] - 0s 2ms/step - loss: 5

1/1 [==============================] - 0s 999us/step - loss: 3.3011e-06
Epoch 197/4200
1/1 [==============================] - 0s 999us/step - loss: 3.3011e-06
Epoch 198/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.3011e-06
Epoch 199/4200
1/1 [==============================] - 0s 4ms/step - loss: 3.3010e-06
Epoch 200/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.3009e-06
Epoch 201/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.3008e-06
Epoch 202/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.3008e-06
Epoch 203/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.3009e-06
Epoch 204/4200
1/1 [==============================] - 0s 4ms/step - loss: 3.3009e-06
Epoch 205/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.3008e-06
Epoch 206/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.3008e-06
Epoch 207/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.

1/1 [==============================] - 0s 2ms/step - loss: 3.2991e-06
Epoch 293/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2991e-06
Epoch 294/4200
1/1 [==============================] - 0s 1000us/step - loss: 3.2991e-06
Epoch 295/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.2990e-06
Epoch 296/4200
1/1 [==============================] - 0s 999us/step - loss: 3.2990e-06
Epoch 297/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2990e-06
Epoch 298/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2990e-06
Epoch 299/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2990e-06
Epoch 300/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2989e-06
Epoch 301/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2989e-06
Epoch 302/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2989e-06
Epoch 303/4200
1/1 [==============================] - 0s 2ms/step - loss: 3

1/1 [==============================] - 0s 999us/step - loss: 3.2970e-06
Epoch 389/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2969e-06
Epoch 390/4200
1/1 [==============================] - 0s 1000us/step - loss: 3.2969e-06
Epoch 391/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2969e-06
Epoch 392/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2969e-06
Epoch 393/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2968e-06
Epoch 394/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.2968e-06
Epoch 395/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2968e-06
Epoch 396/4200
1/1 [==============================] - 0s 999us/step - loss: 3.2968e-06
Epoch 397/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.2967e-06
Epoch 398/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.2967e-06
Epoch 399/4200
1/1 [==============================] - 0s 999us/step - los

...

1/1 [==============================] - 0s 999us/step - loss: 3.6442e-06
Epoch 3770/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.6440e-06
Epoch 3771/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.6438e-06
Epoch 3772/4200
1/1 [==============================] - 0s 1ms/step - loss: 3.6436e-06
Epoch 3773/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.6434e-06
Epoch 3774/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6432e-06
Epoch 3775/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6430e-06
Epoch 3776/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.6428e-06
Epoch 3777/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6426e-06
Epoch 3778/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6424e-06
Epoch 3779/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6422e-06
Epoch 3780/4200
1/1 [==============================] - 0s 2ms/step -

1/1 [==============================] - 0s 999us/step - loss: 3.6251e-06
Epoch 3865/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6249e-06
Epoch 3866/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6247e-06
Epoch 3867/4200
1/1 [==============================] - 0s 1000us/step - loss: 3.6245e-06
Epoch 3868/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.6243e-06
Epoch 3869/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.6241e-06
Epoch 3870/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.6239e-06
Epoch 3871/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6237e-06
Epoch 3872/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6235e-06
Epoch 3873/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6233e-06
Epoch 3874/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6231e-06
Epoch 3875/4200
1/1 [==============================] - 0s 2ms/ste

1/1 [==============================] - 0s 2ms/step - loss: 3.6053e-06
Epoch 3960/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6051e-06
Epoch 3961/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6048e-06
Epoch 3962/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6046e-06
Epoch 3963/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6044e-06
Epoch 3964/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6042e-06
Epoch 3965/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6040e-06
Epoch 3966/4200
1/1 [==============================] - 0s 5ms/step - loss: 3.6038e-06
Epoch 3967/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6036e-06
Epoch 3968/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6033e-06
Epoch 3969/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.6031e-06
Epoch 3970/4200
1/1 [==============================] - 0s 2ms/step - l

1/1 [==============================] - 0s 3ms/step - loss: 4.9552e-06
Epoch 4055/4200
1/1 [==============================] - 0s 2ms/step - loss: 9.4668e-06
Epoch 4056/4200
1/1 [==============================] - 0s 2ms/step - loss: 7.0848e-06
Epoch 4057/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.5823e-06
Epoch 4058/4200
1/1 [==============================] - 0s 3ms/step - loss: 6.0893e-06
Epoch 4059/4200
1/1 [==============================] - 0s 2ms/step - loss: 7.5342e-06
Epoch 4060/4200
1/1 [==============================] - 0s 2ms/step - loss: 4.5398e-06
Epoch 4061/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.9359e-06
Epoch 4062/4200
1/1 [==============================] - 0s 2ms/step - loss: 6.1782e-06
Epoch 4063/4200
1/1 [==============================] - 0s 2ms/step - loss: 5.5778e-06
Epoch 4064/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.6415e-06
Epoch 4065/4200
1/1 [==============================] - 0s 1ms/step - l

1/1 [==============================] - 0s 2ms/step - loss: 3.5527e-06
Epoch 4150/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.5523e-06
Epoch 4151/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.5519e-06
Epoch 4152/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.5518e-06
Epoch 4153/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.5517e-06
Epoch 4154/4200
1/1 [==============================] - 0s 1000us/step - loss: 3.5514e-06
Epoch 4155/4200
1/1 [==============================] - 0s 1ms/step - loss: 3.5511e-06
Epoch 4156/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.5509e-06
Epoch 4157/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.5508e-06
Epoch 4158/4200
1/1 [==============================] - 0s 3ms/step - loss: 3.5505e-06
Epoch 4159/4200
1/1 [==============================] - 0s 2ms/step - loss: 3.5502e-06
Epoch 4160/4200
1/1 [==============================] - 0s 3ms/step 

In [17]:
all_scores

[]

In [18]:
all_mse

[]

In [19]:
tstF0 = prd[0]
tstF1 = prd[1]
tstF2 = prd[2]
tstF2.shape

(16, 2)

In [20]:
fold0 = labels[0]
fold1 = labels[1]
fold2 = labels[2]
type (fold1)
fold2.shape

(16, 2)

In [21]:
predF0 = min_max_scaler_Target.inverse_transform(tstF0)
predF1 = min_max_scaler_Target.inverse_transform(tstF1)
predF2 = min_max_scaler_Target.inverse_transform(tstF2)

In [22]:
predT0 = min_max_scaler_Target.inverse_transform(fold0)
predT1 = min_max_scaler_Target.inverse_transform(fold1)
predT2 = min_max_scaler_Target.inverse_transform(fold2)

In [23]:
error0 = predT0-predF0
error1 = predT1-predF1
error2 = predT2-predF2

In [28]:
df = pd.DataFrame(predT2)
df.to_csv('fo.csv', index=False)

In [24]:
print ('error in fold 1**************')
print (error0)
print ('error in fold 2**************')
print (error1)
print ('error in fold 3**************')
print (error2)

error in fold 1**************
[[ 0.0125  -0.2    ]
 [ 0.025    0.8    ]
 [-0.04375  0.     ]
 [ 0.01875 -0.6    ]
 [ 0.      -0.4    ]
 [-0.05    -0.5    ]
 [ 0.04375  0.1    ]
 [ 0.075   -0.2    ]
 [ 0.025   -0.3    ]
 [ 0.0375  -0.4    ]
 [-0.05    -0.6    ]
 [ 0.0375  -0.3    ]
 [ 0.01875 -0.1    ]
 [-0.05     0.3    ]
 [-0.0125   0.2    ]
 [-0.04375  0.3    ]]
error in fold 2**************
[[-0.0375    0.      ]
 [ 0.       -0.4     ]
 [ 0.05625   0.      ]
 [-0.0125   -0.2     ]
 [-0.03125   0.7     ]
 [-0.01875   0.      ]
 [-0.065625 -0.5     ]
 [-0.01875  -0.4     ]
 [-0.05625   0.7     ]
 [-0.00625   0.3     ]
 [ 0.01875  -0.1     ]
 [-0.025    -0.1     ]
 [ 0.025    -0.1     ]
 [-0.00625  -0.2     ]
 [-0.0125    0.5     ]
 [-0.09375   0.4     ]]
error in fold 3**************
[[ 0.025    -0.3     ]
 [-0.01875   0.4     ]
 [-0.028125  0.3     ]
 [-0.05      0.1     ]
 [ 0.025     0.      ]
 [-0.05625  -0.2     ]
 [ 0.1125   -0.5     ]
 [ 0.09375   0.      ]
 [ 0.05     -0.7    

In [25]:
preds = models.predict(X_train)
print ()

In [26]:
#error in training
preds_proj = min_max_scaler_Target.inverse_transform(preds)
t = y_trainB-preds_proj
print (t)


           0    1
0   0.012500 -0.2
1   0.025000  0.8
2  -0.043750  0.0
3   0.018750 -0.6
4   0.000000 -0.4
5  -0.050000 -0.5
6   0.043750  0.1
7   0.075000 -0.2
8   0.056250 -0.3
9   0.037500 -0.4
10 -0.050000 -0.6
11  0.037500 -0.3
12  0.018750 -0.1
13 -0.050000  0.3
14 -0.012500  0.2
15 -0.043750  0.3
16 -0.006250  0.0
17  0.000000 -0.4
18  0.056250  0.0
19 -0.012500 -0.2
20 -0.031250  0.7
21 -0.018750  0.0
22 -0.065625 -0.5
23  0.012500 -0.4
24 -0.025000  0.7
25 -0.006250  0.3
26  0.018750 -0.1
27  0.006250 -0.1
28  0.025000 -0.1
29  0.009375 -0.2
30  0.018750  0.5
31 -0.062500  0.4
32  0.025000 -0.3
33 -0.018750  0.4
34 -0.028125  0.3
35 -0.050000  0.1
36  0.025000  0.0
37 -0.056250 -0.2
38  0.112500 -0.5
39  0.093750  0.0
40  0.050000 -0.7
41  0.050000 -0.1
42  0.062500  0.0
43 -0.056250  0.0
44  0.012500 -0.5
45  0.056250  0.0
46  0.050000  0.4
47 -0.043750 -0.3


In [27]:

import math
from sklearn.metrics import mean_squared_error
mse_true  = mean_squared_error(y_trainB, preds_proj)
math.sqrt(mse_true)

0.2565996962968292

In [28]:
mse_true

0.06584340413962499

In [29]:
import math
from sklearn.metrics import mean_squared_error
mse_true_1, mse_true_2, mse_true_3  = (mean_squared_error(predT0, predF0), mean_squared_error(predT1, predF1),
                                     mean_squared_error(predT2, predF2)) 
                                   
mse_true_1, mse_true_2, mse_true_3 

(0.07668579106675225, 0.0682656859863778, 0.052790832533639795)

In [30]:
rmse1, rmse2, rmse3  = (math.sqrt(mse_true_1), math.sqrt(mse_true_2), math.sqrt(mse_true_3))
rmse1, rmse2, rmse3 

(0.2769219945521703, 0.2612770291977039, 0.22976255685737787)

In [31]:

#X_test = test_data
predst = models.predict(X_test)

In [32]:
#error in testing
preds_projT = min_max_scaler_Target.inverse_transform(predst)
error = y_testB-preds_projT
print (error)

          0    1
0  -0.01875  0.4
1  -0.02500  0.3
2  -0.02500  0.0
3  -0.02500 -0.6
4   0.03125 -0.5
5  -0.00625 -0.7
6   0.02500  0.2
7  -0.06875 -0.2
8  -0.01250 -0.4
9   0.00000  0.1
10 -0.01250  0.2
11 -0.03125  0.1
12  0.06875  0.2
13  0.06875  0.1
14  0.06250 -0.1
15 -0.03125  0.2
16  0.02500 -0.3


In [33]:

import math
from sklearn.metrics import mean_squared_error
mse_true  = mean_squared_error(y_testB, preds_projT)
math.sqrt(mse_true)

0.2341972363960075

In [34]:
import math
from sklearn.metrics import mean_squared_error
mse_true  = mean_squared_error(y_testB, preds_projT)
mse_true

0.05484834553552743

In [38]:
t

,0,1
0,0.012500,-0.2
1,0.025000,0.8
2,-0.043750,0.0
3,0.018750,-0.6
4,0.000000,-0.4
5,-0.050000,-0.5
6,0.043750,0.1
7,0.075000,-0.2
8,0.056250,-0.3
9,0.037500,-0.4


In [41]:
df = pd.DataFrame(error)

In [42]:
df = pd.DataFrame(error)
df.to_csv('fold0.csv', index=False)

In [43]:
df

,0,1
0,-0.01875,0.4
1,-0.02500,0.3
2,-0.02500,0.0
3,-0.02500,-0.6
4,0.03125,-0.5
5,-0.00625,-0.7
6,0.02500,0.2
7,-0.06875,-0.2
8,-0.01250,-0.4
9,0.00000,0.1
